In [47]:
import polars as pl
import polars.selectors as cs

Part 1

In [ ]:
# from io import StringIO
# IN_ = """123 328  51 64 
#  45 64  387 23 
#   6 98  215 314
# *   +   *   +"""
# IN = StringIO(IN_)
IN = "d6_in.txt"
(
  pl.read_csv(IN, has_header=False)
  .with_columns(
    pl.col("column_1")
    .str
    .strip_chars(" ")
    .str
    .replace_all(r"\s+", ",")
    .str
    .split(",")
    .list
    .to_struct(),
  )
  .unnest("column_1")
  .reverse()
  .transpose()
  .rename({"column_0": "ops"})
  .with_columns(cs.contains("column_").cast(pl.Int64))
  .with_columns(
    total=
    pl.when(pl.col("ops") == "+")
    .then(pl.reduce(function=lambda acc, x: acc + x, exprs=cs.contains("column_")))
    .otherwise(pl.reduce(function=lambda acc, x: acc * x, exprs=cs.contains("column_")))
  )
  .select(pl.col("total").sum())
)

total
i64
5877594983578
